In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_adapter import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

/root/MetaLifelongLanguage/env-adapter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
# model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-02-13-srev-curri/OML-order1-id4-2023-02-13_05-46-13.147873.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-23-lorav2/OML-order1-id4-2023-02-23_23-07-12.342518.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr001-2023-02-26-lorav6/OML-order1-id4-2023-02-27_01-31-30.024416.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2023-02-27-lorav7/OML-order1-id4-2023-02-28_01-16-48.273716.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497510_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2023-02-27-lorav7/OML-order1-id4-2023-02-28_01-16-48.273739_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 1e-3*2,
    "inner_adapter_lr": 1e-4*20,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 8,
    "updates": 5*1,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc",
    "adapter": True
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
lora                     bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               1


2023-02-28 06:35:29,521 - OML-Log - INFO - Loaded TransformerRLN as RLN
2023-02-28 06:35:29,527 - OML-Log - INFO - Loaded LinearPLN as PLN
/root/MetaLifelongLanguage/env-adapter/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.train()
    learner.pln.train()
    # Enable adapter training (Freeze all Others that isn't LoRA)
    learner.rln.encoder.train_adapter("lora")

    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.combinedRLNPLN, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=True) as (frlnpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                output = frlnpln(input_dict)
                loss = learner.loss_fn(output, labels)
                # This here is changed!!
                diffopt.step(loss, grad_callback=lambda grads: [g.detach() if g is not None else None for g in grads])
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_output = frlnpln(query_input_dict) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/950
Support set metrics: Loss = 0.5471, accuracy = 0.8750, precision = 0.8436, recall = 0.8200, F1 score = 0.8276
Query set metrics: Loss = 0.7001, accuracy = 0.5000, precision = 0.6667, recall = 0.6667, F1 score = 0.5000
Query ID 1/950
Support set metrics: Loss = 0.5529, accuracy = 0.7750, precision = 0.8133, recall = 0.8000, F1 score = 0.8002
Query set metrics: Loss = 0.5232, accuracy = 0.7500, precision = 0.7917, recall = 0.7500, F1 score = 0.7417
Query ID 2/950
Support set metrics: Loss = 0.5315, accuracy = 0.8250, precision = 0.8829, recall = 0.8400, F1 score = 0.8492
Query set metrics: Loss = 0.9422, accuracy = 0.6250, precision = 0.7000, recall = 0.7000, F1 score = 0.6667
Query ID 3/950
Support set metrics: Loss = 0.5594, accuracy = 0.7500, precision = 0.7631, recall = 0.7600, F1 score = 0.7496
Query set metrics: Loss = 0.7796, accuracy = 0.8750, precision = 0.8333, recall = 0.9167, F1 score = 

Support set metrics: Loss = 0.5704, accuracy = 0.7000, precision = 0.7450, recall = 0.6800, F1 score = 0.6983
Query set metrics: Loss = 0.4373, accuracy = 0.8750, precision = 0.7500, recall = 0.7500, F1 score = 0.7500
Query ID 36/950
Support set metrics: Loss = 0.6438, accuracy = 0.7250, precision = 0.7035, recall = 0.6800, F1 score = 0.6854
Query set metrics: Loss = 0.5743, accuracy = 0.6250, precision = 0.5833, recall = 0.6250, F1 score = 0.5417
Query ID 37/950
Support set metrics: Loss = 0.4878, accuracy = 0.7750, precision = 0.8219, recall = 0.7800, F1 score = 0.7832
Query set metrics: Loss = 0.9349, accuracy = 0.6250, precision = 0.4000, recall = 0.4167, F1 score = 0.3875
Query ID 38/950
Support set metrics: Loss = 0.5908, accuracy = 0.8250, precision = 0.8655, recall = 0.8200, F1 score = 0.8319
Query set metrics: Loss = 1.5148, accuracy = 0.3750, precision = 0.2917, recall = 0.2917, F1 score = 0.2917
Query ID 39/950
Support set metrics: Loss = 0.4620, accuracy = 0.8250, precision

Support set metrics: Loss = 0.5156, accuracy = 0.8250, precision = 0.8514, recall = 0.8600, F1 score = 0.8484
Query set metrics: Loss = 0.7792, accuracy = 0.6250, precision = 0.7222, recall = 0.6667, F1 score = 0.6889
Query ID 72/950
Support set metrics: Loss = 0.7474, accuracy = 0.8250, precision = 0.8956, recall = 0.8400, F1 score = 0.8651
Query set metrics: Loss = 0.6183, accuracy = 0.7500, precision = 0.6250, recall = 0.6250, F1 score = 0.5833
Query ID 73/950
Support set metrics: Loss = 0.5538, accuracy = 0.7750, precision = 0.8296, recall = 0.7800, F1 score = 0.7772
Query set metrics: Loss = 1.0390, accuracy = 0.5000, precision = 0.4667, recall = 0.4333, F1 score = 0.3933
Query ID 74/950
Support set metrics: Loss = 0.5498, accuracy = 0.8000, precision = 0.8800, recall = 0.8400, F1 score = 0.8401
Query set metrics: Loss = 0.6106, accuracy = 0.6250, precision = 0.7500, recall = 0.6667, F1 score = 0.6583
Query ID 75/950
Support set metrics: Loss = 0.5276, accuracy = 0.7250, precision

Support set metrics: Loss = 0.5908, accuracy = 0.7250, precision = 0.7468, recall = 0.7200, F1 score = 0.7217
Query set metrics: Loss = 0.9405, accuracy = 0.5000, precision = 0.3333, recall = 0.3333, F1 score = 0.3200
Query ID 107/950
Support set metrics: Loss = 0.6152, accuracy = 0.7750, precision = 0.8551, recall = 0.7800, F1 score = 0.7853
Query set metrics: Loss = 1.9402, accuracy = 0.3750, precision = 0.2000, recall = 0.4000, F1 score = 0.2667
Query ID 108/950
Support set metrics: Loss = 0.5774, accuracy = 0.6750, precision = 0.6668, recall = 0.6400, F1 score = 0.6412
Query set metrics: Loss = 1.8936, accuracy = 0.3750, precision = 0.3667, recall = 0.4000, F1 score = 0.3333
Query ID 109/950
Support set metrics: Loss = 0.5209, accuracy = 0.8000, precision = 0.8644, recall = 0.8400, F1 score = 0.8276
Query set metrics: Loss = 0.8575, accuracy = 0.7500, precision = 0.7917, recall = 0.7917, F1 score = 0.7917
Query ID 110/950
Support set metrics: Loss = 0.6490, accuracy = 0.7500, preci

Support set metrics: Loss = 0.5645, accuracy = 0.7500, precision = 0.8016, recall = 0.7800, F1 score = 0.7790
Query set metrics: Loss = 0.8551, accuracy = 0.5000, precision = 0.5000, recall = 0.4583, F1 score = 0.4750
Query ID 142/950
Support set metrics: Loss = 0.5133, accuracy = 0.7750, precision = 0.7969, recall = 0.7800, F1 score = 0.7757
Query set metrics: Loss = 0.5709, accuracy = 0.7500, precision = 0.6875, recall = 0.6250, F1 score = 0.6310
Query ID 143/950
Support set metrics: Loss = 0.4867, accuracy = 0.8250, precision = 0.8568, recall = 0.8000, F1 score = 0.7870
Query set metrics: Loss = 0.6261, accuracy = 0.5000, precision = 0.4375, recall = 0.4375, F1 score = 0.4375
Query ID 144/950
Support set metrics: Loss = 0.7426, accuracy = 0.7250, precision = 0.7637, recall = 0.7800, F1 score = 0.7542
Query set metrics: Loss = 1.5793, accuracy = 0.3750, precision = 0.3333, recall = 0.4167, F1 score = 0.3333
Query ID 145/950
Support set metrics: Loss = 0.5488, accuracy = 0.8500, preci

...

Support set metrics: Loss = 0.5106, accuracy = 0.8750, precision = 0.8976, recall = 0.8750, F1 score = 0.8784
Query set metrics: Loss = 0.7747, accuracy = 0.8750, precision = 1.0000, recall = 0.9000, F1 score = 0.9333
Query ID 794/950
Support set metrics: Loss = 0.3740, accuracy = 0.9500, precision = 0.9542, recall = 0.9500, F1 score = 0.9497
Query set metrics: Loss = 1.2888, accuracy = 0.3750, precision = 0.7500, recall = 0.4375, F1 score = 0.5167
Query ID 795/950
Support set metrics: Loss = 0.5121, accuracy = 0.8750, precision = 0.8914, recall = 0.8750, F1 score = 0.8731
Query set metrics: Loss = 0.4070, accuracy = 0.7500, precision = 0.8333, recall = 0.8333, F1 score = 0.8333
Query ID 796/950
Support set metrics: Loss = 0.5643, accuracy = 0.8250, precision = 0.8986, recall = 0.8250, F1 score = 0.8292
Query set metrics: Loss = 1.2367, accuracy = 0.6250, precision = 0.6000, recall = 0.6667, F1 score = 0.5667
Query ID 797/950
Support set metrics: Loss = 0.4776, accuracy = 0.9500, preci

Support set metrics: Loss = 0.4833, accuracy = 0.9000, precision = 0.9185, recall = 0.9000, F1 score = 0.9011
Query set metrics: Loss = 0.1487, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 829/950
Support set metrics: Loss = 0.4603, accuracy = 0.9250, precision = 0.9271, recall = 0.9250, F1 score = 0.9217
Query set metrics: Loss = 0.4515, accuracy = 0.8750, precision = 0.8571, recall = 0.8571, F1 score = 0.8571
Query ID 830/950
Support set metrics: Loss = 0.6419, accuracy = 0.7750, precision = 0.8879, recall = 0.7750, F1 score = 0.7798
Query set metrics: Loss = 0.1841, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 831/950
Support set metrics: Loss = 0.5316, accuracy = 0.8500, precision = 0.9073, recall = 0.8500, F1 score = 0.8500
Query set metrics: Loss = 0.3362, accuracy = 0.8750, precision = 0.8333, recall = 0.8333, F1 score = 0.8333
Query ID 832/950
Support set metrics: Loss = 0.5586, accuracy = 0.8750, preci

Support set metrics: Loss = 0.5066, accuracy = 0.8500, precision = 0.9281, recall = 0.8500, F1 score = 0.8732
Query set metrics: Loss = 0.4325, accuracy = 0.8750, precision = 1.0000, recall = 0.8750, F1 score = 0.9167
Query ID 864/950
Support set metrics: Loss = 0.5600, accuracy = 0.8750, precision = 0.9143, recall = 0.8750, F1 score = 0.8875
Query set metrics: Loss = 1.1921, accuracy = 0.7500, precision = 0.5833, recall = 0.6667, F1 score = 0.6111
Query ID 865/950
Support set metrics: Loss = 0.5518, accuracy = 0.8500, precision = 0.9125, recall = 0.8500, F1 score = 0.8705
Query set metrics: Loss = 0.4365, accuracy = 0.8750, precision = 1.0000, recall = 0.9000, F1 score = 0.9333
Query ID 866/950
Support set metrics: Loss = 0.4718, accuracy = 0.8750, precision = 0.9018, recall = 0.8750, F1 score = 0.8687
Query set metrics: Loss = 0.9043, accuracy = 0.7500, precision = 0.9167, recall = 0.8333, F1 score = 0.8611
Query ID 867/950
Support set metrics: Loss = 0.5329, accuracy = 0.8500, preci

Support set metrics: Loss = 0.5724, accuracy = 0.8750, precision = 0.9323, recall = 0.8750, F1 score = 0.8903
Query set metrics: Loss = 0.7339, accuracy = 0.7500, precision = 0.9000, recall = 0.8333, F1 score = 0.8267
Query ID 899/950
Support set metrics: Loss = 0.5114, accuracy = 0.8500, precision = 0.8875, recall = 0.8500, F1 score = 0.8622
Query set metrics: Loss = 1.1556, accuracy = 0.6250, precision = 0.7000, recall = 0.6667, F1 score = 0.6333
Query ID 900/950
Support set metrics: Loss = 0.5166, accuracy = 0.8250, precision = 0.8906, recall = 0.8250, F1 score = 0.8086
Query set metrics: Loss = 0.9406, accuracy = 0.5000, precision = 0.4762, recall = 0.5714, F1 score = 0.5000
Query ID 901/950
Support set metrics: Loss = 0.6136, accuracy = 0.8750, precision = 0.9115, recall = 0.8750, F1 score = 0.8720
Query set metrics: Loss = 0.7428, accuracy = 0.7500, precision = 0.8333, recall = 0.7500, F1 score = 0.7778
Query ID 902/950
Support set metrics: Loss = 0.4616, accuracy = 0.8750, preci

Support set metrics: Loss = 0.5827, accuracy = 0.8750, precision = 0.9194, recall = 0.8750, F1 score = 0.8837
Query set metrics: Loss = 0.4450, accuracy = 0.7500, precision = 0.6667, recall = 0.7143, F1 score = 0.6857
Query ID 934/950
Support set metrics: Loss = 0.5101, accuracy = 0.8250, precision = 0.8830, recall = 0.8250, F1 score = 0.8458
Query set metrics: Loss = 0.8166, accuracy = 0.7500, precision = 0.5952, recall = 0.7143, F1 score = 0.6381
Query ID 935/950
Support set metrics: Loss = 0.6393, accuracy = 0.8250, precision = 0.8823, recall = 0.8250, F1 score = 0.8334
Query set metrics: Loss = 0.4925, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 936/950
Support set metrics: Loss = 0.4755, accuracy = 0.9000, precision = 0.9375, recall = 0.9000, F1 score = 0.9069
Query set metrics: Loss = 0.4189, accuracy = 0.8750, precision = 0.8333, recall = 0.8333, F1 score = 0.8333
Query ID 937/950
Support set metrics: Loss = 0.4148, accuracy = 0.9250, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-inlr002-2023-02-27-lorav7/OML-order1-id4-2023-02-28_01-16-48.273716_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-inlr002-2023-02-27-lorav7/OML-order1-id4-2023-02-28_01-16-48.273716_update5_time_inference.csv
